In [91]:
import wandb
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D

In [92]:
!pip show wandb

Name: wandb
Version: 0.15.4
Summary: A CLI and library for interacting with the Weights and Biases API.
Home-page: https://github.com/wandb/wandb
Author: Weights & Biases
Author-email: support@wandb.com
License: MIT license
Location: c:\users\janos.laptop-42clk60g\anaconda3\envs\ds\lib\site-packages
Requires: appdirs, Click, docker-pycreds, GitPython, pathtools, protobuf, psutil, PyYAML, requests, sentry-sdk, setproctitle, setuptools
Required-by: 


In [161]:
api = wandb.Api()

FONT_SIZE = 12

metric_name = "val_loss"
metric_label = "Validation Loss"
entity = "janousy"
project = "fedstellar-cifar10-lf-targeted"
# group = "FASHIONMNIST_1_MLP_Sentinel_fully_ModelPoisoning_0_N50-S0_R80_salt_20230710_084210"
SAVE_PATH = os.path.join(os.getcwd(), "tmp_figures")
print("Writing figure to: " + SAVE_PATH)
file_name = metric_label

isExist = os.path.exists(SAVE_PATH)
if not isExist:
    os.makedirs(SAVE_PATH)


#'attack_env.poisoned_sample_percent': 100,


Writing figure to: C:\Users\janos.LAPTOP-42CLK60G\Repos\fedstellar-robust\evaluation\metrics\resources\tmp_figures


In [162]:
aggregation_list = ["Krum", "FedAvg", "TrimmedMean", "FlTrust", "Sentinel", "SentinelGlobal"]
metrics_list = ['CPU', 'Memory', 'Traffic']
final_metrics = pd.DataFrame(data=None, index=aggregation_list, columns=metrics_list, dtype=None, copy=None)
final_metrics_error = pd.DataFrame(data=None, index=aggregation_list, columns=metrics_list, dtype=None, copy=None)

# filter by attack
runs = api.runs(f'{entity}/{project}', filters={'config.attack_env.poisoned_node_percent': 80,
                                                'config.attack_env.poisoned_sample_percent': 100,
                                                'description': 'participant_0',
                                                })
assert len(runs) == 6
for run in runs:
    print(run.description)
    print(run.group)
    # cpu_util = run.scan_history(keys=["CPU Utilization (%)"])
    # history = run.scan_history(keys=metric_keys)
    system_metrics = run.history(stream="events")
    mean_all_cpus = system_metrics[(list(system_metrics.filter(regex='system.gpu.*.cpu_percent')))].mean()
    max_cpu = mean_all_cpus.max()
    min_cpu = mean_all_cpus.min()
    mean_cpu = mean_all_cpus.min()
    err_cpu = (max_cpu - min_cpu) / 2
    mean_ram = system_metrics['system.proc.memory.rssMB'].mean()
    max_ram = system_metrics['system.proc.memory.rssMB'].max()
    min_ram = system_metrics['system.proc.memory.rssMB'].min()
    err_ram = (max_ram - min_ram) / 2
    bytes_sent = system_metrics['system.network.sent'].mean()
    agg_metrics = [mean_cpu, mean_ram, bytes_sent]
    agg_error = [err_cpu, err_ram, 0]
    agg = run.config['aggregator_args']['algorithm']
    final_metrics.loc[agg] = agg_metrics
    final_metrics_error.loc[agg] = agg_error

participant_0
CIFAR10_1_simplemobilenet_SentinelGlobal_fully_LabelFlipping_1_N80-S100_R0_salt_20230714_135825
participant_0
CIFAR10_1_simplemobilenet_Sentinel_fully_LabelFlipping_1_N80-S100_R0_salt_20230712_212847
participant_0
CIFAR10_1_simplemobilenet_FlTrust_fully_LabelFlipping_1_N80-S100_R0_salt_20230712_194716
participant_0
CIFAR10_1_simplemobilenet_TrimmedMean_fully_LabelFlipping_1_N80-S100_R0_salt_20230712_180422
participant_0
CIFAR10_1_simplemobilenet_FedAvg_fully_LabelFlipping_1_N80-S100_R0_salt_20230712_160915
participant_0
CIFAR10_1_simplemobilenet_Krum_fully_LabelFlipping_1_N80-S100_R0_salt_20230712_142751


In [163]:
system_metrics

,system.network.sent,system.gpu.0.powerPercent,system.network.recv,system.gpu.1.temp,system.cpu.9.cpu_percent,system.disk,system.cpu.1.cpu_percent,system.gpu.0.powerWatts,system.cpu.7.cpu_percent,_wandb,...,system.cpu.0.cpu_percent,system.cpu.13.cpu_percent,system.proc.memory.percent,system.gpu.1.powerWatts,system.cpu.10.cpu_percent,system.gpu.1.powerPercent,system.gpu.0.memoryAllocated,_timestamp,system.cpu.8.cpu_percent,system.cpu.14.cpu_percent
0,82544.87,11.97,93828.80,49.40,2.07,53.8,8.69,38.31,2.76,True,...,2.98,1.73,1.13,23.90,2.14,7.47,2.18,1.689165e+09,1.84,2.44
1,236948.73,11.94,252480.00,49.00,2.13,53.8,1.80,38.21,2.50,True,...,1.93,1.87,1.13,23.98,1.93,7.49,2.18,1.689165e+09,1.97,3.03
2,5236540.27,40.10,468932.87,49.07,59.35,53.8,52.19,128.15,56.65,True,...,54.74,56.01,2.37,25.30,53.65,7.91,30.69,1.689165e+09,53.05,56.13
3,6840296.00,58.91,1978343.40,49.00,77.51,53.8,74.94,188.70,74.84,True,...,74.57,73.73,3.44,25.10,72.70,7.84,49.53,1.689165e+09,73.17,74.94
4,11234473.40,66.07,6083440.47,49.00,86.68,53.8,86.74,211.44,86.46,True,...,85.77,86.89,3.47,25.35,86.34,7.92,53.70,1.689165e+09,84.67,87.15
5,14929983.73,60.53,10211303.07,49.00,68.00,53.8,63.60,193.68,69.43,True,...,65.56,66.41,3.47,25.04,66.69,7.83,45.40,1.689165e+09,64.80,64.88
6,17925308.73,67.26,12609679.80,49.00,84.31,53.8,83.09,215.23,83.85,True,...,83.51,82.99,3.48,25.47,81.68,7.96,51.21,1.689165e+09,83.53,81.32
7,22335678.27,67.00,17088251.00,49.20,73.71,53.8,73.39,214.40,72.95,True,...,76.77,75.39,3.48,25.29,75.11,7.90,51.03,1.689165e+09,72.73,75.13
8,26026454.47,61.69,20479838.20,50.00,70.43,53.8,68.55,197.40,69.57,True,...,69.05,69.97,3.48,25.09,67.29,7.84,45.63,1.689165e+09,69.23,67.99
9,29374512.80,64.72,22879239.47,50.00,87.50,53.8,89.46,207.06,87.24,True,...,87.35,87.33,3.48,25.47,85.38,7.96,54.45,1.689165e+09,86.69,89.31


In [168]:
import numpy as np

gpu = system_metrics[(list(system_metrics.filter(regex='system.gpu.0.gpu$')))]
gpu = gpu.dropna() # remove system errors
# gpu = gpu.values.tolist()

gpu.mean().values.tolist()[0]


AttributeError: 'DataFrame' object has no attribute 'setm'

In [169]:
gpu.sem().values.tolist()[0]

7.318377214930811

In [ ]:
max_gpu = np.max(gpu)
min_gpu = np.min(gpu)
mean_gpu = np.mean(gpu)

In [158]:
import numpy as np

max_gpu = np.max(gpu)
min_gpu = np.min(gpu)
# mean_gpu = mean(gpu.values.tolist())
mean_gpu = np.mean(gpu)
range_gpu = (max_gpu - min_gpu) / 2
mean_ram = system_metrics['system.gpu.0.memoryAllocated'].mean() / 100 * 10000
max_ram = system_metrics['system.gpu.0.memoryAllocated'].max() / 100 * 10000
min_ram = system_metrics['system.gpu.0.memoryAllocated'].min() / 100 * 10000
range_ram = (max_ram - min_ram) / 2
bytes_sent_mb = system_metrics['system.network.sent'].mean() / pow(1000,2)
agg_metrics = [mean_gpu, mean_ram, bytes_sent_mb]
agg_range = [range_gpu, range_ram, 0]

agg_range

[48.265, 2711.4999999999995, 0]

In [119]:
for col in final_metrics.columns:
    ax = final_metrics.plot.bar(y='CPU Max', rot=0)
    fig = ax.get_figure()
    fig.show()
    plt.savefig(os.path.join(SAVE_PATH, file_name), dpi=600, bbox_inches='tight')

C:\Users\janos.LAPTOP-42CLK60G\AppData\Local\Temp\ipykernel_23008\2334910246.py:4: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
C:\Users\janos.LAPTOP-42CLK60G\AppData\Local\Temp\ipykernel_23008\2334910246.py:4: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
C:\Users\janos.LAPTOP-42CLK60G\AppData\Local\Temp\ipykernel_23008\2334910246.py:4: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


In [120]:
df = pd.DataFrame({'lab':['A', 'B', 'C'], 'val':[10, 30, 20]})
ax = df.plot.bar(x='lab', y='val', rot=0)
plt.savefig(os.path.join(SAVE_PATH, file_name), dpi=600, bbox_inches='tight')